In [101]:
import numpy as np
import itertools as it

import allel
import pandas as pd

from _plotly_future_ import v4_subplots
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
init_notebook_mode(connected=True)
    
from datetime import datetime

import collections
def recursively_default_dict():
    return collections.defaultdict(recursively_default_dict)

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

## The coalescent and sampling.


### Requirements

    - effective population size per generation;
    - number of generations;
    - scaled mutation rate.
    
   
### Index

#### I. Model assumptions.

#### II. Coalescent simulations.

#### III. Deployment across models. 

#### IV. Variance analysis. 


### I. Model assumptions.

**Demographics**

We will deal with terminal branches only. 

We're going to go with populations of species X, for which models have been proposed.
see model dict below. `pop` dictionaries contain model parameters per population specifc branches:

- `N`: Population effective size;
- `T`: Branch length


In [102]:
model_dict= {
        'schweinfurthii': {
            'N':10528, 
            'T':17925
        },
        'troglodytes': {
            'N': 72001, 
            'T': 17925
            },
        'ellioti': {
            'N': 6033,
            'T': 33402
        },
        'verus': {
            'N': 5710,
            'T': 33402
        }
}

**mutations** 

We re going to refer here to coalescent theory. Our objective will not concern statistics on observed data, but simulations. For this we will rely on the notebook [Simulations](https://nbviewer.jupyter.org/github/SantosJGND/Coalescent/blob/master/Simulations.ipynb), section **Algorithms for simulating sequence evolution**,  from this [repository](https://github.com/SantosJGND/Coalescent) focused on coalescent explorations in python.


To keep it simple we will consider in this notebook a single simulation. 

Given :
- time;
- effective population size;
- scaled mutation rate;

we draw the number of expected mutations emerging in our branch of interest until the present. 

## II. Simulations. Coalescent given number of samples.

We use a procedure of coalescent simulation given the number of samples. We extract per simulation the number of mutations simulated that are more recent than the MRCA of this branch. See the notebook above for a visual inspection of the output of the simulation procedure. We will use the function SimIII from that notebook, modified here for our purposes. 

We first chose a model from the dictionary above. 


In [103]:
pop_select= 'schweinfurthii'

We begin with a simple example: the number of private mutations expected expected in a sample of size `k` of haplotypes of size `seqL`, given an estimate of mutation rate per generation per base pair `muNuc`.

From this input we calculate the parameters `mu`, the expected mutation rate per generation and `theta`, the Watterson estimator of population genetic diversity. 

In [104]:
muNuc= 1.08e-8
seqL= 1e6

mu= muNuc * seqL
Ne= model_dict[pop_select]["N"]
k= 100 # Number of samples
Theta= 4 * Ne * mu


We now perform coalescent simulations. This returns a network containing coalescent and mutation events from present haplotypes to their MRCA (function `SimIII`). Each event has an associated time stamp. We use the function `get_PA` to extract the time stamps of mutation events more recent than the branch of interest. 

In [105]:
from tools.coal_sims import SimIII, get_PA

sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)

branch_len= model_dict[pop_select]['T']
MRCA= branch_len / Ne / 2

PA_obtain= get_PA(leaves,Ne,MRCA)
print('number of private alleles obtained: {}'.format(len(PA_obtain)))

number of private alleles obtained: 2282


#### get results across sample sizes

We now move to obtain results for a range of sample sizes. 

A number of repeats is performed per sample size, simulations are probablistic.

In [106]:
Nreps= 50
sample_range= [2,500]
Nsteps= 20

range_use= np.linspace(*sample_range,Nsteps)
Nstudy_dict= {int(z):[] for z in range_use}

for si in range_use:
    k= int(si)
    for nr in range(Nreps):
        
        sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)
        
        PA_obtain= get_PA(leaves,Ne,MRCA)
        
        Nstudy_dict[k].append(len(PA_obtain))
        

The number of private alleles sampled is plotted against sample size. 

In [107]:
stats_dict= {
    z: {
        'mean': np.mean(g),
        'std': np.std(g)
    } for z,g in Nstudy_dict.items()
}
samp_order= sorted(Nstudy_dict.keys())

fig= [go.Scatter(
    x= samp_order,
    y= [stats_dict[x]['mean'] for x in samp_order],
    error_y= dict(
        array= [stats_dict[x]['std'] for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    )
)]

layout= go.Layout(
    title= pop_select,
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= 'private alleles'
    )
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)

### III. Deployment. Across populations:

We now scale this approach to cover all branches in `model_dict`. 

**common parameters**

In [110]:
### biological parameters
muNuc= 1.08e-8
seqL= 1e6
mu= muNuc * seqL

## sampling parameters
Nreps= 10
sample_range= [2,100]
Nsteps= 30
range_use= np.linspace(*sample_range,Nsteps)

**deployment**

In [111]:
pops_dict= {}
study_dict= {}

for pop_select in model_dict.keys():
    Ne= model_dict[pop_select]["N"]
    
    Theta= 4 * Ne * mu
    
    branch_len= model_dict[pop_select]['T']
    MRCA= branch_len / Ne / 2

    Nstudy_dict= {int(z):[] for z in range_use}

    for si in range_use:
        k= int(si)
        for nr in range(Nreps):

            sim_keys, leaves, edges= SimIII(k= k, Theta= Theta)

            PA_obtain= get_PA(leaves,Ne,MRCA)

            Nstudy_dict[k].append(len(PA_obtain))
    
    stats_dict= {
        z: {
            'mean': np.mean(g),
            'std': np.std(g)
        } for z,g in Nstudy_dict.items()
    }
    samp_order= sorted(Nstudy_dict.keys())
    
    pops_dict[pop_select]= stats_dict
    study_dict[pop_select]= Nstudy_dict


**plot**

In [112]:
fig= [go.Scatter(
    x= samp_order,
    y= [pops_dict[pop][x]['mean'] for x in samp_order],
    error_y= dict(
        array= [pops_dict[pop][x]['std'] for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    name= pop
) for pop in model_dict.keys()]

layout= go.Layout(
    title= 'pop comparison',
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= '# of private alleles'
    ),
    height= 950,
    width= 950
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)

### III. Variance in count / frequency statistics.

Some statistics of interest can depend on the number of mutations found in a sample. 

This is the case if mutations are characterised in any way, creating factors. If we are interested in ascertaining the true value of the size of each class in a population, than the variance in our observations are dependent on the number of samples. 

We investigate the impact of sampling haplotypes from model populations on estimator variance. 

We are going to assume that count numbers per sample size behave normally and extract posterior distributions from our simulated data this way. 


We begin by considering the variance around a single class. We set the expected proportion of that class in final counts to be `Pc`. We assum the proportion of cases of this class sampled will behave as a binomial variable of mean equals `Pc`, `n`  the sample size.


We extract two statistics:

- variance in proportion: `var_calc` and `var_samp`, obtained analytically and through sampling with `Nreps_II` replicates (bellow) under the assumptions stated above respectively.

- differences to expecation: `diffs`, the difference between expected and observed proportions when sampling.


#### i. Calc

Using data sampled above.

In [113]:
from scipy.stats import norm
from scipy.stats import binom

Pc= 1 / 96
Nreps_II= 10

syndicate_dict= {
    z: {k: {} for k in g.keys()} for z,g in study_dict.items()
}

for pop_select in study_dict.keys():
    
    for si,sim_extract in study_dict[pop_select].items():
        size= int(si)
        
        pop_loc= pops_dict[pop_select][si]['mean']
        pop_scale= pops_dict[pop_select][si]['std']
        
        ## here the normal assumption is made
        sim_extract= norm.rvs(loc= pop_loc,scale= pop_scale,size= Nreps)
        sim_extract= np.array(sim_extract,dtype= int)
        sim_extract= sim_extract[sim_extract > 0]
        varA= []
        store_var= []
        store_diffs= []
        
        for nr in sim_extract:
            ## extract expected variance in proportion
            var_calc= nr * Pc * (1 - Pc)
            var_calc= np.sqrt(var_calc) / nr
            
            ## sample proportions assuming binomial behavior
            var_samp= binom.rvs(nr,Pc,size= Nreps_II)
            prop_vec= var_samp / nr
            
            ## proportion std and difference to expectation.
            prop_var= np.std(prop_vec)
            prop_diffs= Pc - prop_vec
            
            varA.append(var_calc)
            store_var.append(prop_var)
            store_diffs.extend(prop_diffs)
        
        syndicate_dict[pop_select][si]= {
            'var_calc': varA,
            'var_samp': store_var,
            'diffs': store_diffs
        }



#### ii. Plot. 

Interactive plots are not readable through NBviewer.

In [114]:
## Var
var_select= 'var_calc'

fig= [go.Scatter(
    x= samp_order,
    y= [np.nanmean(syndicate_dict[pop][x][var_select]) for x in samp_order],
    error_y= dict(
        array= [np.nanstd(syndicate_dict[pop][x][var_select]) for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    name= pop
) for pop in syndicate_dict.keys()]

layout= go.Layout(
    title=  'stat = {}'.format(var_select),
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= var_select
    )
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)

In [115]:
## Var
var_select= 'var_samp'

fig= [go.Scatter(
    x= samp_order,
    y= [np.nanmean(syndicate_dict[pop][x][var_select]) for x in samp_order],
    error_y= dict(
        array= [np.nanstd(syndicate_dict[pop][x][var_select]) for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    name= pop
) for pop in syndicate_dict.keys()]

layout= go.Layout(
    title=  'stat = {}'.format(var_select),
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= var_select
    )
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)

In [116]:
## Var
var_select= 'diffs'

fig= [go.Scatter(
    x= samp_order,
    y= [np.nanmean(syndicate_dict[pop][x][var_select]) for x in samp_order],
    error_y= dict(
        array= [np.nanstd(syndicate_dict[pop][x][var_select]) for x in samp_order],
        type= 'data',
        #symmetric= True,
        visible=True
    ),
    name= pop
) for pop in syndicate_dict.keys()]

layout= go.Layout(
    title=  'stat = {}'.format(var_select),
    xaxis= dict(
        title= 'Nsamp'
    ),
    yaxis= dict(
        title= var_select
    )
)

Figure= go.Figure(data=fig, layout= layout)
iplot(Figure)